In [4]:
# 3.0 LOAD PART 1 AND PART 2 AS DATAFRAMES (df_1 and df_2)
import json
import logging
import os
import traceback
import requests
import pprint as pp
import pandas as pd

# API: https://datausa.io/api/data?drilldowns=Nation&measures=Population
# Documentation: https://datausa.io/about/api/

C:\Users\sdgur\anaconda3\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [5]:
# df_1
file_headers = [ "series_id", "year",  "period", "value", "footnote_codes" ]
file =  "pr.data.0.Current.csv"
current = pd.read_csv(file)
df_1 = pd.read_csv(file, sep='\t', skiprows=(1), names=file_headers)

# file2 = "pr.data.0.Current.xls"
# df_1 = pd.read_csv(file2, sep='\t', skiprows=(1), names=file_headers)
# df_1.sample
df_1

,series_id,year,period,value,footnote_codes
0,PRS30006011,1995,Q01,2.600,NaN
1,PRS30006011,1995,Q02,2.100,NaN
2,PRS30006011,1995,Q03,0.900,NaN
3,PRS30006011,1995,Q04,0.100,NaN
4,PRS30006011,1995,Q05,1.400,NaN
...,...,...,...,...,...
33871,PRS88003203,2021,Q04,120.007,NaN
33872,PRS88003203,2021,Q05,116.110,NaN
33873,PRS88003203,2022,Q01,123.905,NaN
33874,PRS88003203,2022,Q02,126.498,R


In [6]:
URL_API = "https://datausa.io/api/data?drilldowns=Nation&measures=Population"
requests_session = requests.Session()
response = requests_session.get(URL_API)
print(response.text)

pp.pprint(response.text)

{"data":[{"ID Nation":"01000US","Nation":"United States","ID Year":2020,"Year":"2020","Population":326569308,"Slug Nation":"united-states"},{"ID Nation":"01000US","Nation":"United States","ID Year":2019,"Year":"2019","Population":324697795,"Slug Nation":"united-states"},{"ID Nation":"01000US","Nation":"United States","ID Year":2018,"Year":"2018","Population":322903030,"Slug Nation":"united-states"},{"ID Nation":"01000US","Nation":"United States","ID Year":2017,"Year":"2017","Population":321004407,"Slug Nation":"united-states"},{"ID Nation":"01000US","Nation":"United States","ID Year":2016,"Year":"2016","Population":318558162,"Slug Nation":"united-states"},{"ID Nation":"01000US","Nation":"United States","ID Year":2015,"Year":"2015","Population":316515021,"Slug Nation":"united-states"},{"ID Nation":"01000US","Nation":"United States","ID Year":2014,"Year":"2014","Population":314107084,"Slug Nation":"united-states"},{"ID Nation":"01000US","Nation":"United States","ID Year":2013,"Year":"201

In [7]:
# df_2
dict_2 = json.loads(response.text)
df_2 = pd.DataFrame(dict_2['data'])

df_2

,ID Nation,Nation,ID Year,Year,Population,Slug Nation
0,01000US,United States,2020,2020,326569308,united-states
1,01000US,United States,2019,2019,324697795,united-states
2,01000US,United States,2018,2018,322903030,united-states
3,01000US,United States,2017,2017,321004407,united-states
4,01000US,United States,2016,2016,318558162,united-states
5,01000US,United States,2015,2015,316515021,united-states
6,01000US,United States,2014,2014,314107084,united-states
7,01000US,United States,2013,2013,311536594,united-states


In [8]:
# 3.1 GENERATE POPULATION STATS
# df_2.loc[df_2['ID Year'] >= 2013, 'Population'].mean()
# mean = df_2.loc[2:7, 'ID Year':'Population'] 
# mean = df_2.loc[2:7, ['ID Year','Population']] 

df_2.loc[2013 <= df_2['ID Year'], 'Population'].mean()
mean = df_2.loc[2:7, 'Population'].mean()
mean
stdev = df_2.loc[2:7, 'Population'].std()
stdev
print(f"mean: {mean}, stdev.s: {stdev}")

mean: 317437383.0, stdev.s: 4257089.5415293295


In [231]:
# 3.2 FIND BEST YEAR
# For every series_id, find the best year: the year with the max/largest sum of "value" for all quarters in that year. 
# Generate a report with each series id, the best year for that series, and the summed value for that year. 
# https://www.makeuseof.com/learn-how-to-create-excel-pivot-tables-in-python/

In [406]:
# Convert df_1 into pivot table.
df_1_pivot = df_1.pivot_table(index = ["year", "series_id", "period"], values=["value"])
df_1_pivot

value
year series_id         period         
1995 PRS30006011       Q01       2.600
                       Q02       2.100
                       Q03       0.900
                       Q04       0.100
                       Q05       1.400
...                                ...
2022 PRS88003202       Q02       8.600
                       Q03       5.100
     PRS88003203       Q01     123.905
                       Q02     126.498
                       Q03     128.078

[33876 rows x 1 columns]

In [10]:
# Remove Q05 rows from pivot table.
# https://www.shanelynn.ie/pandas-drop-delete-dataframe-rows-columns/
df_1_clean = df_1.loc[df_1["period"] != "Q05"]
df_1_clean

,series_id,year,period,value,footnote_codes
0,PRS30006011,1995,Q01,2.600,NaN
1,PRS30006011,1995,Q02,2.100,NaN
2,PRS30006011,1995,Q03,0.900,NaN
3,PRS30006011,1995,Q04,0.100,NaN
5,PRS30006011,1996,Q01,-0.200,NaN
...,...,...,...,...,...
33870,PRS88003203,2021,Q03,116.951,NaN
33871,PRS88003203,2021,Q04,120.007,NaN
33873,PRS88003203,2022,Q01,123.905,NaN
33874,PRS88003203,2022,Q02,126.498,R


In [213]:
# Validate number of removed rows with Excel
# Note: first column of df_1_clean is not the correct row number because rows were removed from the original dataframe df_1
print(f"df_1.shape: {df_1.shape}, df_1_clean.shape: {df_1_clean.shape}, confirmed via Excel 7569 rows deleted")


df_1.shape: (33876, 5), df_1_clean.shape: (26307, 5), confirmed via Excel 7569 rows deleted


In [12]:
df_1_clean_pivot = df_1_clean.pivot_table(index=["year", "series_id", "period"], values=["value"])
df_1_clean_pivot

value
year series_id         period         
1995 PRS30006011       Q01       2.600
                       Q02       2.100
                       Q03       0.900
                       Q04       0.100
     PRS30006012       Q01       1.500
...                                ...
2022 PRS88003202       Q02       8.600
                       Q03       5.100
     PRS88003203       Q01     123.905
                       Q02     126.498
                       Q03     128.078

[26307 rows x 1 columns]

In [13]:
# Sum values for all quarters, groupby series_id, year. 
df_1_final = df_1_clean_pivot.pivot_table(index=["year", "series_id", "period"], values=["value"], aggfunc=sum)
df_1_final
groups = df_1_final.groupby(['series_id', 'year'])
period_sums = groups.sum()
period_sums

value
series_id         year         
PRS30006011       1995    5.700
                  1996   -0.400
                  1997    3.500
                  1998    3.400
                  1999   -6.100
...                         ...
PRS88003203       2018  441.277
                  2019  451.557
                  2020  454.241
                  2021  464.441
                  2022  378.481

[6636 rows x 1 columns]

In [15]:
# Find max sum of values per series_id
max_values = groups.sum().pivot_table(index=["series_id"], values=["value"], aggfunc=[max])
max_values

,max
,value
series_id,
PRS30006011,13.300
PRS30006012,11.600
PRS30006013,586.675
PRS30006021,14.100
PRS30006022,8.900
...,...
PRS88003192,262.700
PRS88003193,517.947


In [203]:

series_ids, years, max_vals, best_years = [], [], [], []

for row in max_values.itertuples():
    ser_id = row[0]
    val = row[1]
    
    for per_row in period_sums.itertuples():
#         display(ser_id)
#         display(val)
        per_ser_id = per_row[0][0]
        year = per_row[0][1]
        max_val = per_row[1]
        if ser_id == per_ser_id and max_val == val:
            series_ids.append(ser_id)
            years.append(year)
            max_vals.append(val)
            
#             print(ser_id, year, max_val)

report_series_ids = pd.Series(series_ids)
report_years = pd.Series(years)
report_max_vals = pd.Series(max_vals)
# report_periods = pd.Series(periods)
# best_years = [report_series_ids, report_years, report_max_vals]

# best_years
dict = {"series_id": report_series_ids, "year": report_years, "value": report_max_vals}
df_best_years = pd.DataFrame(dict)
df_best_years
# df = pd.DataFrame(lst, columns =['Name', 'Age'])
# df
# df_best_years = pd.DataFrame(best_years, columns=["series_id", "year", "value"])


# list of name, degree, score
# nme = ["aparna", "pankaj", "sudhir", "Geeku"]
# deg = ["MBA", "BCA", "M.Tech", "MBA"]
# scr = [90, 40, 80, 98]
  
# # dictionary of lists 
# dict = {'name': nme, 'degree': deg, 'score': scr} 
    
# df = pd.DataFrame(dict)
    
# df 
# display(max_values)
# display(period_sums)
# print('sean')

,series_id,year,value
0,PRS30006011,2022,13.300
1,PRS30006012,2021,11.600
2,PRS30006013,1998,586.675
3,PRS30006021,2010,14.100
4,PRS30006022,2010,8.900
...,...,...,...
232,PRS88003192,2002,262.700
233,PRS88003193,2021,517.947
234,PRS88003201,2022,29.600
235,PRS88003202,2022,27.300


In [221]:
# df_new.head(30)
# 3.3 VALUE FOR SERIES_ID 

from pyspark.sql.functions import trim
# dfb = df_best_years
# display(df_2)
# df_2[df_2['ID Year'] == 2020]
# display(dfb)
# dfb[dfb['value'] == 13.3]
# dfb[dfb['year'] == 2021]
# dfb[dfb['series_id'] == "PRS30006032"]
# display(df_1_clean)
# df_1_clean_pivot = df_1_clean.pivot_table(index=["year", "series_id", "period"], values=["value"])
# display(df_1_clean[df_1_clean['period'] == 'Q01', df_1_clean['series_id'] == 'PRS30006011'])



,series_id,year,period,value,footnote_codes


In [405]:
import math

def trim_all_columns(dfb):
    """
    Trim whitespace from ends of each value across all series in dataframe
    """
    trim_strings = lambda x: x.strip() if isinstance(x, str) else x
    return dfb.applymap(trim_strings)

# simple example of trimming whitespace from data elements
# df = pd.DataFrame([['  a  ', 10, ' b ', 20], ['  c  ', 5, ' d ', 7], ['e', 6, 'f ', 8]])
# df = trim_all_columns(df)
df3 = trim_all_columns(df_1_clean)
df4 = trim_all_columns(df_best_years)

# display(df3)
# df3.loc[['series_id'] == "PRS30006032"]
# display(df4[df4['value'] == 27.8])
# print(df4.columns)
# [x for x in df4.columns if x.endswith(' ') or x.startswith(' ')]
# df4.columns = df4.columns.str.strip()
df3.columns = df3.columns.str.strip()
# display(df3.columns)
# display(df3[(df3['series_id']=="PRS30006032") & (df3['period']=='Q01') & (df3['footnote_codes'] != None])
df3_clean = df3[(df3['series_id']=="PRS30006032") & (df3['period']=='Q01')  ]
# display(df3_clean)
# df2 =   df3[(df3['Fee']>=22000              ) & (df3['Discount']< 3000) & df3['Courses'].str.startswith('P')][['Courses','Fee', 'Discount']] 
# df3_clean = df3.pivot_table(index=["series_id", "year", "period"], values=["value"])
df3_clean_trim = trim_all_columns(df3_clean)
df_left = df3_clean_trim.drop('footnote_codes', axis=1)
dfb = df_left.sort_values('value', ascending=False).head(1)
display(dfb.style.hide_index())
display(df_2)


# dfa = df_left.loc[df_left['value'].idxmax()]
# dfb = dfa.to_frame()
# dfb.columns = dfb.iloc[1]
# display(dfb)

# dfb.columns=df.iloc[0]

series_id,year,period,value
PRS30006032,2022,Q01,7.300000


,ID Nation,Nation,ID Year,Year,Population,Slug Nation
0,01000US,United States,2020,2020,326569308,united-states
1,01000US,United States,2019,2019,324697795,united-states
2,01000US,United States,2018,2018,322903030,united-states
3,01000US,United States,2017,2017,321004407,united-states
4,01000US,United States,2016,2016,318558162,united-states
5,01000US,United States,2015,2015,316515021,united-states
6,01000US,United States,2014,2014,314107084,united-states
7,01000US,United States,2013,2013,311536594,united-states


In [256]:
import math

math.isnan(math.inf)

False

In [32]:
import json
import logging
import boto3
import os
import traceback
import urllib3
import requests

logger = logging.getLogger()
logger.setLevel(logging.INFO)

def lambda_handler(event, context):
    
    print(event)
    logger.info(event)
    
    http = urllib3.PoolManager()
    s3_client = boto3.client('s3')
    BLS_URL = os.getenv("BLS_URL")
    
    
    try:
        requests_session = requests.Session()
        response = requests_session.get(BLS_URL)
        
        print(response)
        
    
    except Exception as e:
        traceback.print_exc()
        # exception_message = "Error"
        # logger.error(e)
    
    
    return {
        'statusCode': 200,
        'body': json.dumps('Hello from Lambda!')
    }


ModuleNotFoundError: No module named 'boto3'

In [ ]:
"""
The parent directory https://download.bls.gov/pub/time.series/ does not always update with the correct last modified date 
of the subdirectories, see "sa" subdirectory as an example, whose most recent last modified date is 2011, not 2010. 


In [34]:
import pprint as pp
txt = '<html><head><title>download.bls.gov - /pub/time.series/pr/</title></head><body><H1>download.bls.gov - /pub/time.series/pr/</H1><hr><pre><A HREF="/pub/time.series/">[To Parent Directory]</A><br><br> 12/7/2022  8:30 AM          111 <A HREF="/pub/time.series/pr/pr.class">pr.class</A><br> 9/13/2022  3:52 PM          562 <A HREF="/pub/time.series/pr/pr.contacts">pr.contacts</A><br> 12/7/2022  8:30 AM      1424071 <A HREF="/pub/time.series/pr/pr.data.0.Current">pr.data.0.Current</A><br> 12/7/2022  8:30 AM      3047695 <A HREF="/pub/time.series/pr/pr.data.1.AllData">pr.data.1.AllData</A><br> 12/7/2022  8:30 AM          214 <A HREF="/pub/time.series/pr/pr.duration">pr.duration</A><br> 12/7/2022  8:30 AM           40 <A HREF="/pub/time.series/pr/pr.footnote">pr.footnote</A><br> 12/7/2022  8:30 AM          609 <A HREF="/pub/time.series/pr/pr.measure">pr.measure</A><br>  1/7/1994  2:53 PM          146 <A HREF="/pub/time.series/pr/pr.period">pr.period</A><br>11/18/2011  3:05 PM           79 <A HREF="/pub/time.series/pr/pr.seasonal">pr.seasonal</A><br> 12/7/2022  8:30 AM          263 <A HREF="/pub/time.series/pr/pr.sector">pr.sector</A><br> 12/7/2022  8:30 AM        15657 <A HREF="/pub/time.series/pr/pr.series">pr.series</A><br>11/17/2011  4:11 PM        18343 <A HREF="/pub/time.series/pr/pr.txt">pr.txt</A><br></pre><hr></body></html>'
print(txt)
# pp.pprint(txt)
txt.index("</A>", 2)
txt[150:185]
txt.find(".", 3)

<html><head><title>download.bls.gov - /pub/time.series/pr/</title></head><body><H1>download.bls.gov - /pub/time.series/pr/</H1><hr><pre><A HREF="/pub/time.series/">[To Parent Directory]</A><br><br> 12/7/2022  8:30 AM          111 <A HREF="/pub/time.series/pr/pr.class">pr.class</A><br> 9/13/2022  3:52 PM          562 <A HREF="/pub/time.series/pr/pr.contacts">pr.contacts</A><br> 12/7/2022  8:30 AM      1424071 <A HREF="/pub/time.series/pr/pr.data.0.Current">pr.data.0.Current</A><br> 12/7/2022  8:30 AM      3047695 <A HREF="/pub/time.series/pr/pr.data.1.AllData">pr.data.1.AllData</A><br> 12/7/2022  8:30 AM          214 <A HREF="/pub/time.series/pr/pr.duration">pr.duration</A><br> 12/7/2022  8:30 AM           40 <A HREF="/pub/time.series/pr/pr.footnote">pr.footnote</A><br> 12/7/2022  8:30 AM          609 <A HREF="/pub/time.series/pr/pr.measure">pr.measure</A><br>  1/7/1994  2:53 PM          146 <A HREF="/pub/time.series/pr/pr.period">pr.period</A><br>11/18/2011  3:05 PM           79 <A HRE

27